<a href="https://colab.research.google.com/github/sjut/DPO_Materials/blob/master/Collocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Коллокации
Одна из задач АОТ - излвечений из коллекции текстов (из большого текста) устойчивых словосочетаний (например, 'бить баклуши' или 'бросить взгляд') или каких-то конструкций ('Торги стартовали с отметки X за баррель марки Brent' или 'Вам дать телефон X?') или имен главных героев ('Шерлок Холмс'). 

Будем использовать два разных набора данных

(1) Возьмем три текста: часть Анны Карениной, часть Бесов, Мертвые души Гоголя. Также возьмем очень условный стоп-лист для русского языка

(2) Возьмем коллекцию текстов про нанотехнологии

In [142]:
#загружаем файлы художественной коллекции
#https://colab.research.google.com/\ - загружаем тексты, которые лежат на гитхабе в папке 
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/anna_karenina_part.txt
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/besy_dostoevsky_part.txt
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/avidreaders.ru__mertvye-dushi.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1350k  100 1350k    0     0  5233k      0 --:--:-- --:--:-- --:--:-- 5233k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1206k  100 1206k    0     0  2594k      0 --:--:-- --:--:-- --:--:-- 2588k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1342k  100 1342k    0     0  5993k      0 --:--:-- --:--:-- --:--:-- 5993k


In [0]:
#создаем список файлов с указанием кодировки (чтобы в дальнейшем не писать вызов функции отдельно, а вызывать функции в цикле)
files = ["anna_karenina_part.txt", "besy_dostoevsky_part.txt", "avidreaders.ru__mertvye-dushi.txt"]
enc = "utf-8"
n_files = len(files)  

In [144]:
#считываем содержимое файлов; "texts" - список, элемнты списка - тексты, которые содержатся в наших файлах
texts = [open(fname, encoding = enc).read() for fname in files]
#смотрим, что записалось в первые 100 символов нулевого элемента списка
texts[0][:100]

'Annotation\n\n\n«Анна Каренина» – это сложное, психологически утонченное, остропроблемное произведение,'

In [145]:
#смотрим длину текстов; перебираем по числу элементов списка "texts"
for i in range(n_files):
  print(f"Длина {files[i]:40} - {len(texts[i]):10} символов")

Длина anna_karenina_part.txt                   -     769151 символов
Длина besy_dostoevsky_part.txt                 -     683391 символов
Длина avidreaders.ru__mertvye-dushi.txt        -     762191 символов


In [185]:
from string import punctuation
import numpy as np
!curl --remote-name \
     -H 'Accept: application/vnd.github.v3.raw' \
     --location https://raw.githubusercontent.com/sjut/DPO_Materials/master/%D0%94%D0%B0%D0%BD%D0%BD%D1%8B%D0%B5%20%D0%B8%20%D1%81%D1%81%D1%8B%D0%BB%D0%BA%D0%B8/stoplist_ru.txt
stoplist_ru = set(open("stoplist_ru.txt", mode = 'r', encoding = "utf-8").read().split())
print(stoplist_ru)

def tokenize(text, del_stop = False):
    tokenized_text = [word.strip(punctuation) for word in text.lower().split()]
                                                            
    tokenized_text = [word for word in tokenized_text if word]
    if del_stop:
      tokenized_text = [word for word in tokenized_text if word not in stoplist_ru]
    return tokenized_text


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    76  100    76    0     0    410      0 --:--:-- --:--:-- --:--:--   413
{'был', 'с', '-', '–', 'была', 'было', 'были', 'то', 'же', 'в', 'и', 'а', 'на', '—', 'не'}


Сравним тексты по словам

In [147]:
stop = True
norm_texts = [tokenize(text, stop) for text in texts]
print(norm_texts[0][100:110])
#смотрим длину текстов в токенах
for i in range(n_files):
  print(f"Длина {files[i]:40} - {len(norm_texts[i]):10} токенов; {len(set(norm_texts[i]))} уникальных словоформ")

['все', 'смешалось', 'доме', 'облонских', 'жена', 'узнала', 'что', 'муж', 'связи', 'бывшею']
Длина anna_karenina_part.txt                   -     103424 токенов; 21101 уникальных словоформ
Длина besy_dostoevsky_part.txt                 -      90743 токенов; 20995 уникальных словоформ
Длина avidreaders.ru__mertvye-dushi.txt        -      99424 токенов; 25889 уникальных словоформ


In [0]:
from collections import Counter

In [159]:
vocabs = [Counter(norm_text) for norm_text in norm_texts]
most_commons = [vocab.most_common(20) for vocab in vocabs]

joint_most_common = {}
for mc in most_commons:
  for i in mc:
    joint_most_common[i[0]] = [0,0,0]

for k in joint_most_common:
  for i in range(n_files):
    joint_most_common[k][i] = vocabs[i].get(k, 0)
  print(f'{k:15} {joint_most_common[k]}')



что             [2620, 1816, 1982]
он              [2317, 1224, 1341]
она             [1537, 562, 138]
я               [1392, 1772, 760]
как             [1201, 808, 1225]
но              [1158, 959, 621]
его             [1088, 653, 733]
это             [1003, 707, 616]
к               [875, 583, 571]
ее              [834, 341, 177]
все             [750, 236, 691]
так             [610, 588, 727]
сказал          [581, 41, 493]
ему             [539, 214, 351]
за              [523, 433, 489]
о               [513, 464, 262]
только          [477, 408, 450]
левин           [476, 0, 0]
ты              [444, 162, 323]
у               [417, 487, 583]
вы              [379, 900, 310]
всё             [0, 509, 63]
бы              [369, 468, 658]
по              [379, 456, 544]
мне             [291, 397, 232]
еще             [284, 383, 350]
чичиков         [0, 0, 602]
да              [353, 329, 565]
из              [329, 330, 433]
уже             [234, 299, 412]


In [164]:
# устанавливаем pymorphy и импортируем необходимые модули
!pip install pymorphy2[fast]
import itertools
import nltk
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
morph = MorphAnalyzer()
nltk.download("stopwords")
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TypeError: ignored

In [179]:
 print(punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [188]:
#сформируем список стопслов, тут можно сделать свой собственный файл стопслов и добавить его
print(stoplist_ru)
stops =  stoplist_ru.union(stopwords.words('russian'))
print(stops)
print(stops.intersection(stoplist_ru))

{'был', 'с', '-', '–', 'была', 'было', 'были', 'то', 'же', 'в', 'и', 'а', 'на', '—', 'не'}
{'теперь', 'может', 'тоже', 'какой', 'много', 'два', 'он', 'тут', 'между', 'зачем', 'себе', 'хорошо', 'чем', 'один', 'во', 'даже', 'себя', 'они', 'надо', 'всех', 'три', 'уж', 'был', 'ним', 'над', 'их', 'или', 'все', 'совсем', 'если', 'ну', 'там', 'ведь', 'чтоб', 'у', 'при', 'хоть', 'к', 'ты', 'меня', 'ли', 'нет', 'здесь', 'ее', 'можно', 'для', 'была', 'ж', 'то', 'эти', '—', 'никогда', 'разве', 'эту', 'со', 'сам', 'тебя', 'под', '–', 'есть', 'было', 'по', 'моя', 'быть', 'за', 'нибудь', 'в', 'на', 'только', 'из', 'но', 'чуть', 'лучше', 'где', 'будет', 'этой', 'им', 'иногда', 'куда', 'вот', 'ему', 'с', 'без', 'как', 'его', 'потом', 'этого', 'такой', 'чего', 'до', 'что', 'я', 'после', 'раз', 'уже', 'нельзя', 'потому', 'вдруг', 'другой', 'тогда', 'ней', 'и', 'когда', 'впрочем', 'о', 'от', 'того', 'наконец', 'него', 'кто', 'через', 'этом', 'еще', 'мой', 'этот', 'она', 'сейчас', 'всего', 'чтобы', 'об', 

In [0]:
#функция для токенизации и нормализации текста
def normalize(text):
    punct = punctuation +"«»"
    normalized_text = [morph.parse(word.strip(punct))[0].normal_form for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word and word not in stops]
    return normalized_text

In [0]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams
    

In [231]:
#лемматизируем тексты
lemm_texts = [normalize(text) for text in texts]
print(lemm_texts[2][:20])

['annotation', 'мёртвый', 'душа', 'уникальный', 'роман', 'стать', 'русский', 'литература', 'своеобразный', 'эталон', 'иронический', 'проза', 'книга', 'раздёргать', 'цитата', 'ещё', 'xix', 'no-прежний', 'потрясать', 'воображение']


In [197]:
word_counter = Counter()

for text in lemm_texts:
    word_counter.update(ngrammer(text, n=2))

word_counter.most_common(30)


[('степан трофим', 331),
 ('варвар пётр', 288),
 ('николай всеволод', 288),
 ('весь это', 227),
 ('алексей александр', 218),
 ('степан аркадьй', 205),
 ('пётр степан', 134),
 ('сказать чичиков', 129),
 ('крайний мера', 102),
 ('павел иван', 97),
 ('самый дело', 91),
 ('это весь', 87),
 ('маврикий николай', 85),
 ('человек который', 84),
 ('сергей иван', 82),
 ('весь равно', 82),
 ('это время', 80),
 ('варвара пётр', 77),
 ('знать это', 75),
 ('говорить это', 75),
 ('дарья александр', 74),
 ('это дело', 69),
 ('друг друг', 67),
 ('сказать левин', 67),
 ('сказать это', 66),
 ('молодая человек', 60),
 ('лизавета николай', 56),
 ('сей пора', 55),
 ('весь ещё', 54),
 ('весь жизнь', 54)]

В списке есть коллокации, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

Самый простой способ - взять количество упоминаний биграма и поделить на сумму количеств упоминаний слов по отдельности.
Можно вычислить коэффициент Дайса

In [0]:
def scorer_simple(word_count_a, word_count_b, bigram_count, *args):
    try:
        score = bigram_count/((word_count_a+word_count_b))
    
    except ZeroDivisionError:
        return 0
    
    return score

In [0]:
#Сделаем функцию, которая будет делать счетчики для слов и биграммов.
def collect_stats(texts):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    unigrams = Counter()
    bigrams = Counter()
    
    for text in texts:
        unigrams.update(text)
        bigrams.update(ngrammer(text, 2))
    
    return unigrams, bigrams

In [230]:
unigrams, bigrams = collect_stats(lemm_texts)
unigrams.most_common(50)
bigrams.most_common(50)

TypeError: ignored

In [0]:
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=5):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigrams[bigram], len_vocab, min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [212]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)
bigram2score.most_common(20)

[('весь это', 113.5),
 ('крайний мера', 51.0),
 ('человек который', 42.0),
 ('сергей иван', 41.0),
 ('весь равно', 41.0),
 ('друг друг', 33.5),
 ('сей пора', 27.5),
 ('весь ещё', 27.0),
 ('весь жизнь', 27.0),
 ('весь свой', 25.5),
 ('прасковья иван', 24.0),
 ('весь весь', 20.5),
 ('черта знать', 20.5),
 ('весь сторона', 18.5),
 ('петра степан', 18.0),
 ('весь время', 17.0),
 ('семён яков', 17.0),
 ('весь город', 15.5),
 ('весь знать', 14.5),
 ('герой наш', 14.0)]

In [0]:
from nltk.collocations import *

In [0]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [241]:
all_texts_lemm = []
for t in lemm_texts:
  all_texts_lemm += t
finder2 = BigramCollocationFinder.from_words(all_texts_lemm)
finder2.nbest(bigram_measures.likelihood_ratio, 50)

[('степан', 'трофим'),
 ('николай', 'всеволод'),
 ('варвар', 'пётр'),
 ('алексей', 'александр'),
 ('степан', 'аркадьй'),
 ('крайний', 'мера'),
 ('павел', 'иван'),
 ('сергей', 'иван'),
 ('маврикий', 'николай'),
 ('пётр', 'степан'),
 ('варвара', 'пётр'),
 ('дарья', 'александр'),
 ('юлий', 'михайло'),
 ('фон', 'лембка'),
 ('сей', 'пора'),
 ('марья', 'тимофей'),
 ('лизавета', 'николай'),
 ('весь', 'равно'),
 ('мёртвый', 'душа'),
 ('семён', 'яков'),
 ('друг', 'друг'),
 ('прасковья', 'иван'),
 ('ваш', 'превосходительство'),
 ('афанасий', 'василий'),
 ('сказать', 'чичиков'),
 ('андрей', 'антоно'),
 ('юлия', 'михайло'),
 ('мадам', 'шталь'),
 ('самый', 'дело'),
 ('молодая', 'человек'),
 ('дарья', 'павло'),
 ('агафья', 'михайло'),
 ('ха', 'ха'),
 ('ваш', 'сиятельство'),
 ('полковой', 'командир'),
 ('петра', 'степан'),
 ('графиня', 'лидий'),
 ('алексей', 'нилыч'),
 ('константин', 'левин'),
 ('константин', 'дмитрич'),
 ('андрей', 'иван'),
 ('матрена', 'филимон'),
 ('весь', 'это'),
 ('выражение', '